# Database parsing

In [164]:
import pandas, numpy as np
import parsingDatabaseUtils, re
import xml, itertools, xml.etree.ElementTree as ET
import tqdm, importlib

In [62]:
casos = pandas.read_csv('Venezolanas/casos.csv', index_col = 0)
pacientes = pandas.read_csv('Venezolanas/pacientes.csv', index_col = 0)
pacientes.index = pacientes.index.map(str)

registros = pandas.read_csv('Venezolanas/registros.csv', index_col = 0)
registros.index = registros.index.map(str)

diagnosis = pandas.read_csv('Venezolanas/diagnosis.csv', index_col = 0)
procedimientos = pandas.read_csv('Venezolanas/procedimientos.csv', index_col = 0)
procedimientos.index = procedimientos.index.map(str)

procedimientosDesc = pandas.read_csv('Venezolanas/procedimientosID.csv', index_col = 0)
registrosByCaso = registros.groupby('Caso')

C:\Users\CMRC\Miniconda3\lib\site-packages\IPython\core\interactiveshell.py:3063: DtypeWarning: Columns (6,27,55) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
C:\Users\CMRC\Miniconda3\lib\site-packages\IPython\core\interactiveshell.py:3063: DtypeWarning: Columns (19) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [63]:
classificationProcedures = {'H0165': 'o', 'H0193': 'o', 'H2120': 'o', 'H2379': 'o', 'H2383': 'o', 'H2386': 'o', 'H2407': 'o', 'H2415': 'o', 'H2595': 'o', 'H2684': 'o', 'H2849': 'o', 'H2852': 'o', 'H2880': 'o', 'H2882': 'o', 'H2884': 'o', 'H2892': 'o', 'H2901': 'o', 'H2904': 'o', 'H2910': 'o', 'H2916': 'o', 'H2959': 'o', 'H2963': 'o', 'H2974': 'a', 'H2975': 'a', 'H3038': 'o', 'H3065': 'o', 'H3066': 'o', 'H3078': 'o', 'H3089': 'p', 'H3092': 'p', 'H3094': 'p', 'H3099': 'a', 'H3100': 'a', 'H3108': 'o', 'H3109': 'o', 'H3111': 'o', 'H3114': 'o', 'H3118': 'o', 'H4421': 'o', 'H4494': 'o', 'H4496': 'o', 'HE020': 'o'}

## Select a case
Identify all  cases with an associated procedure

In [64]:
# Identify all  cases with an associated procedure

In [66]:
interventionToCase = {}
caseToProcedureBirths = {}
for i,r  in tqdm.tqdm_notebook(registros.iterrows()):
    if isinstance(r.RegistroXML, str) and '<row NombreCampo="IdDescripcion"' in r.RegistroXML:
        et = ET.fromstring(r.RegistroXML)
        idDescripcionProcedimiento = et.find('.//row[@NombreCampo="IdDescripcion"]').get('ValorCampo')
        interventionToCase[idDescripcionProcedimiento] = (r.Caso, r.NumeroHistoria, i)
        procedureType = re.findall('<idProcedimiento>([a-zA-Z0-9]*)</idProcedimiento>', procedimientos.loc[idDescripcionProcedimiento].XmlDescripcion)[0]
        if classificationProcedures[procedureType] == 'p':
            if r.Caso in caseToProcedureBirths:
                print('Error: two procedures in same case', r.Caso, procedureType)
            caseToProcedureBirths[r.Caso] = procedimientos.loc[idDescripcionProcedimiento]

Error: two procedures in same case AD342920 H3094
Error: two procedures in same case AD346540 H3089
Error: two procedures in same case AD350190 H3094
Error: two procedures in same case AD351571 H3094
Error: two procedures in same case AD350755 H3094
Error: two procedures in same case AD353436 H3094
Error: two procedures in same case AD353902 H3089
Error: two procedures in same case AD354284 H3089
Error: two procedures in same case AD356781 H3089
Error: two procedures in same case AD359075 H3094
Error: two procedures in same case NN H3089
Error: two procedures in same case NN H3089
Error: two procedures in same case AD365893 H3094
Error: two procedures in same case AD366060 H3089
Error: two procedures in same case AD363741 H3089
Error: two procedures in same case AD363741 H3089



In [67]:
importlib.reload(parsingDatabaseUtils)

<module 'parsingDatabaseUtils' from 'C:\\Users\\CMRC\\toolsCMRC\\parsingDatabaseUtils.py'>

In [195]:
class BirthDataset:
    """
    Data structure with the information regarding a case, including:
    
    - Procedure
    - Epicrisis
    - Registro de urgencias
    - newborn registration / updates
    - newborn discharge
    """
    posibilitiesAlta = ['alta', 'salida', 'egreso', 'remision', 'traslado']
    def __init__(self, caso, procedure, registros, pacientes):
        self.casoID = caso
        self.procedure = procedure
        #Split between mother and newborn
        self.registersMother = []
        self.registersNewborn = []
        self.registersUnassigned = []
        self.epicrisis = None
        self.ingreso = None
        self.motherData = pacientes.loc[str(registros.iloc[0].NumeroHistoria)]
        
        #Assign each register to mother/newborn or unknown
        for rId, reg in registros.iterrows():
            patient = parsingDatabaseUtils.isMaternalRegister(reg, registros)
            if patient == 'mother':
                self.registersMother.append(reg)
            elif patient == 'newborn':
                self.registersNewborn.append(reg)
            #Ignore the ones that are registro de incapacidad (#46), or a link to the description of a procedure(#145)
            elif reg.CodigoRegistro not in [145, 46]:
                #print(caso, reg.Asunto)
                self.registersUnassigned.append(reg)
            if 'Epicrisis' in str(reg.Asunto):
                self.epicrisis = reg
            if 'Ingreso de Urgencias' == reg.Asunto:
                self.ingreso = reg

        #Find discharge of baby
        self.newbornDischarge = list(filter(lambda s:
                                       any([p in parsingDatabaseUtils.remove_diacritics(str(s.Asunto)).lower()
                                            for p in BirthDataset.posibilitiesAlta]),
                                       self.registersNewborn
                                      )
                                  )
        if self.newbornDischarge:
            self.newbornDischarge = self.newbornDischarge[0]
            
        elif len(self.newbornDischarge) == 0 and len(self.registersNewborn) > 1: 
            i = np.argmax([p.FechaAsignacionRegistro for p in self.registersNewborn])
            self.newbornDischarge = self.registersNewborn[i]
            
    def getNivelN1(self):
        
        etIngreso
        
        res = {}
        res['VAR_0006'] = self.motherData.FechaNac
        #Etnia
        if self.motherData.Etnia == 1:
            res['VAR_0011'] = 'B'
        #Raizal, palenquero, negros/mulatos
        elif self.motherData.Etnia in [3,4,5]:   
            res['VAR_0011'] = 'D'
        #Otras etnias?
        elif self.motherData.Etnia in [2]:
            res['VAR_0011'] = 'E'
            
        #Estudios y alfabetiacion
        if self.motherData.Escolaridad in [2, 3, 4,5,6]:
            res['VAR_0012'] = 'B'
        elif self.motherData.Escolaridad in [1, 8]:
            res['VAR_0012'] = 'A'
            
        #TODO: Que hacer con pre-escolar? 
        if self.motherData.Escolaridad in [8, 1]:
            res['VAR_0013'] = 'A'
        elif self.motherData.Escolaridad in [3]:
            res['VAR_0013'] = 'B'
        elif self.motherData.Escolaridad in [4, 5]:
            res['VAR_0013'] = 'C'
        elif self.motherData.Escolaridad in [6]:
            res['VAR_0013'] = 'D'
            
        if self.motherData.EstadoCivil in ['Casado']:
            res['VAR_0015'] = 'A'
        elif self.motherData.EstadoCivil in ['Soltero']:
            res['VAR_0015'] = 'C'
        elif parsingDatabaseUtils.remove_diacritics(
            self.motherData.EstadoCivil) in ['Union Libre']:
            res['VAR_0015'] = 'D'
            
        res['VAR_0018'] = 'CMRC'
        res['VAR_0019'] = self.motherData.Identificacion 
        
        # Antecedentes
        if self.ingreso is not None:
            etIngreso = ET.fromstring(self.ingreso.RegistroXML)
            
            #Antecedentes familiares
            aFamiliares = parsingDatabaseUtils.findInXML("AntecedentesFamiliares", etIngreso)
            antecedentes = parsingDatabaseUtils.pareAntecendentes(aFamiliares)
            if ('None' in antecentes and len(antecentes) > 1) or len(antecentes) == 0:
                """
                Something weird happened
                """
                pass
            else:
                res['Var_0020'] = 'B' if 'TBC' in antecedentes else 'A'
                res['Var_0022'] = 'B' if 'Diabetes' in antecedentes else 'A'
                res['Var_0024'] = 'B' if 'HTA' in antecedentes else 'A'
                res['Var_0026'] = 'B' if 'Preeclampsia' in antecedentes else 'A'
                res['Var_0028'] = 'B' if 'Eclampsia' in antecedentes else 'A'
                res['Var_0030'] = 'B' if 'Otros' in antecedentes else 'A'

            #Personales solo si no hay nada
            
            #TODO: a bit of parsing could be done, but I do not have time
            res['Var_0055'] = int(parsingDatabaseUtils.findInXML("Peso", etIngreso))
            res['Var_0056'] = float(parsingDatabaseUtils.findInXML("Talla", etIngreso)) * 100 - 100
            
            
        # G P C A : Double check, sometimes it is wrong
        
        #FUM
        
        #Echos
        
        #VIH
        
        #VDRL - RPR

In [ ]:
def getParto

def getNewborn

In [198]:
parsingDatabaseUtils.prettyPrintXML(p.ingreso.RegistroXML)

<?xml version="1.0" ?>
<C Asunto="Ingreso de Urgencias" Caso="AD182404" CentroA="01" CodigoDiagnosticoRelacionado1="Z359" CodigoDiagnosticoRelacionado2="Z353" CodigoDiagnosticoRelacionado3="" Diagnostico="Z358" IdPaciente="152609" Modulo="03" Padre="NULL" PlanAdm="118" Prestador="73572518" Raiz="NULL" Registro="07">
	<row NombreCampo="TipoConsulta" NombreTabla="CamposTexto" ValorCampo="OBSTETRICA"/>
	<row NombreCampo="PlanAdministradora" NombreTabla="CamposTexto" ValorCampo="PARTICULAR / PARTICULARES"/>
	<row NombreCampo="MotivoConsulta" NombreTabla="CamposTexto" ValorCampo="&quot; TENGO CONTRACCIONES&quot;

"/>
	<row NombreCampo="EnfermedadActual" NombreTabla="CamposTexto" ValorCampo="PACIENTE DE 20 A&#209;OS DE EDAD CON CUADRO CLINICO DE 3 DIAS DE EVOLUCION CONISISTENTE EN DOLOR TIPO CONTRACCIONES EN HIPOGASTRIO QUE SE IRRADIAN A ZONA LUMBAR. SIN NINGUNA OTRA SINTOMATOLOGIA APARENTE"/>
	<row NombreCampo="FrecuenciaCardiaca" NombreTabla="CamposReal" ValorCampo="78"/>
	<row NombreCampo

In [194]:
registrosByCaso.get_group(c).iloc[0].NumeroHistoria

numpy.int64

In [196]:
# Split in newborn
registrosByCaso = registros.groupby('Caso')
processedDatasets = {}
for c, p in caseToProcedureBirths.items():
    processedDatasets[c] = BirthDataset(c, p, registrosByCaso.get_group(c), pacientes)

In [200]:
noEpicrsis = []
noIngreso = []

noDischarge = []
for c, p in processedDatasets.items():
    if len(p.registersNewborn) == 0:
        noDischarge.append(c)
    if p.ingreso is None:
        noIngreso.append(c)
print(len(noIngreso), len(noDischarge), len(processedDatasets))

83 58 2044


In [204]:
parsingDatabaseUtils.prettyPrintXML(registros.loc['1030990', 'RegistroXML'])

<?xml version="1.0" ?>
<C Asunto="Epicrisis de la Historia Notas de Ingreso a Piso" Caso="AD284225" CentroA="01" CodigoDiagnosticoRelacionado1="Z392" CodigoDiagnosticoRelacionado2="" CodigoDiagnosticoRelacionado3="" Diagnostico="O829" IdPaciente="184931" Modulo="01" Padre="AD284225" PlanAdm="100" Prestador="9068204" Raiz="01" Registro="450">
	<row NombreCampo="Antecedentes" NombreTabla="CamposTexto" ValorCampo=""/>
	<row NombreCampo="AntecedentesHTML" NombreTabla="CamposTexto" ValorCampo="- DESCRIPCI&#211;N DE LA NOTA:
PACIENTE FEMENINA DE 30 A&#209;OS DE EDAD CON DIAGNOSTICOS DE:- POP DE CESAREA TRANSPERITONEAL(12-04-2018- 3:55PM)- PUERPERIO MEDIATO- RECIEN NACIDO UNICO VIVO(MASCULINO, 3060GR, 50CM)
SUBJETIVO:PACIENTE REFIERE SENTIRSE BIEN, NIEGA FIEBRE, DOLOR, SANGRADO POR GENITALES U OTRA SINTOMATOLOGIA ASOCIADAOBJETIVO:PA: 120/80 MMHG FC: 90 LPM FR: 18 LPM T: 36.5C&#176;PACIENTE CON BUENAS CONDICIONES GENERALES, NORMOCEFALO, CON BUENA IMPLANTACION CAPILAR, PUPILAS ISOCORICAS, NORMO

In [206]:
registrosByCaso.get_group(noIngreso[15])

,NumeroHistoria,Caso,CodigoRegistro,Asunto,FechaAsignacionRegistro,Raiz,Padre,Usuario,Prestador,AdmPlan,...,AltaMedica,IdRegistroPreanestesico,IdRegistroIntraOperatorio,IdRegistroPostOperatorio,Modulo,ProgramaPyP,FechaInicioAtencion,UnidadFuncional,ConfirmacionGuardado,Sincronizado
Id,,,,,,,,,,,,,,,,,,,,,
1230990,206132,AD336659,94,Notas de Ingreso a Piso,2019-04-30 09:02:52.183,NaN,NaN,NCARMONA,45470969,100,...,False,NaN,NaN,NaN,1.0,NaN,2019-04-30 09:02:00.000,NaN,False,NaN
1230994,206132,AD336659,165,Registro del recién nacido,2019-04-30 09:13:07.670,NaN,Padre,sa,1140866516,100,...,False,NaN,NaN,NaN,3.0,NaN,NaN,NaN,False,NaN
1230998,206132,AD336659,145,NaN,2019-04-30 09:16:59.217,NaN,NaN,MBANQUEZ,1128057935,100,...,False,NaN,NaN,NaN,1.0,NaN,NaN,NaN,False,NaN
1231417,206132,AD336659,94,Notas de Ingreso a Piso,2019-04-30 20:38:29.947,NaN,NaN,JPEREZM,9159174,100,...,False,NaN,NaN,NaN,1.0,NaN,2019-04-30 20:36:00.000,NaN,False,NaN
1231551,206132,AD336659,106,Evoluci&#243;n ALTA MEDICA,2019-05-01 08:58:47.423,94.0,1231417,HCASTRO,7883096,100,...,True,NaN,NaN,NaN,1.0,NaN,2019-05-01 08:52:00.000,NaN,False,NaN
1231558,206132,AD336659,450,Epicrisis de la Historia Notas de Ingreso a Piso,2019-05-01 09:06:20.150,1.0,AD336659,HCASTRO,7883096,100,...,False,NaN,NaN,NaN,1.0,NaN,NaN,NaN,False,NaN
1231554,206132,AD336659,46,Registro de Incapacidad,2019-05-01 09:02:55.387,NaN,NaN,HCASTRO,7883096,100,...,False,NaN,NaN,NaN,1.0,NaN,NaN,NaN,False,NaN
1231594,206132,AD336659,166,ALTA DE RN,2019-05-01 09:48:22.140,165.0,1230994,HCASTRO,7883096,100,...,False,NaN,NaN,NaN,1.0,NaN,2019-05-01 09:45:00.000,NaN,False,NaN


In [168]:
processedDatasets[c].epicrisis

In [174]:
c = list(processedDatasets.keys())[0]
parsingDatabaseUtils.prettyPrintXML(processedDatasets[c].epicrisis.RegistroXML)

<?xml version="1.0" ?>
<C Asunto="Epicrisis de la Historia Ingreso de Urgencias" Caso="AD182404" CentroA="01" CodigoDiagnosticoRelacionado1="Z359" CodigoDiagnosticoRelacionado2="" CodigoDiagnosticoRelacionado3="" Diagnostico="Z358" IdPaciente="152609" Modulo="01" Padre="AD182404" PlanAdm="118" Prestador="9068204" Raiz="01" Registro="450">
	<row NombreCampo="Antecedentes" NombreTabla="CamposTexto" ValorCampo=""/>
	<row NombreCampo="AntecedentesHTML" NombreTabla="CamposTexto" ValorCampo="- PERSONALES:
  7-&amp;gt; MENMARQUIA: 12 
VIDA SEXUAL: 16
VIDA OBSTETRICA: 18 
CICLOS IRREGULAR
FUM: NO RECUERDA
 G: 2  P: 1 A: 0  C: O
COMPA&#209;EROS SEXUALES: 1
PLANIFICACION: NO
CONTROL PRENATAL: 0
TIPO DE SANGRE AB+

ECOGRAFIA DE 25/08/16 QUE REPORTA EMBARAZO DE 34 SEMANAS PARA HOY 34,2 SEM
- FAMILIARES:
  HIPERTENSION (PADRE)
- AL&#201;RGICOS:
  
- QUIR&#218;RGICOS:"/>
	<row NombreCampo="AplicaCuidadoEnfermeria" NombreTabla="CamposBoolean" ValorCampo="false"/>
	<row NombreCampo="AsuntoHistoriaPadr